# CS4248 Project Group 23

In [ ]:
# If you wish to run this on Google Colab, mount the Google drive by running this cell or click the `files` icon on the left navbar
# and click mount Google Drive (it takes some time to load)
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/<The path to this notebook in your Google Drive>"
!cd "/content/drive/My Drive/<The path to this notebook in your Google Drive>"

In [1]:
import pandas as pd
from sklearn.metrics import f1_score
import numpy as np

In [2]:
# Unzip raw_data.zip locally
import zipfile
with zipfile.ZipFile('raw_data.zip', 'r') as zip_ref:
    zip_ref.extractall()

Feature Engineering: Capture various features of the text (e.g. punctuation, stopwords, statement length). 
Test out different tokenizers to capture their performance.


In [3]:
import gensim.downloader as api

def load_glove_model():
    glove_model = api.load('glove-wiki-gigaword-300')
    print("Done.",len(glove_model)," words loaded!")
    return glove_model, glove_model.vector_size

glove_model, glove_dim = load_glove_model()

Done. 400000  words loaded!


In [4]:
import re
import string
from nltk.tokenize import word_tokenize

# todo parallelize this in future


from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

def preprocess_text(text):
    # Lowercase text
    text = text.lower()
    # Remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # tokenize
    words = word_tokenize(text)
    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Apply lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    # Convert list of words back to string
    text = ' '.join(words)
    return text

In [6]:
train = pd.read_csv("./raw_data/fulltrain.csv")

# Preprocess all documents in the corpus
preprocessed_corpus = [preprocess_text(text) for text in train.iloc[:, 1]]

# Fit the TF-IDF vectorizer on the preprocessed corpus
vectorizer.fit(preprocessed_corpus)

# Now you can use the fitted vectorizer to transform any document:
def vectorize_text(preprocessed_text):
    vectors = vectorizer.transform([preprocessed_text])
    return vectors.toarray()

x_train = np.array([vectorize_text(text) for text in preprocessed_corpus]) 
x_train = x_train.reshape(x_train.shape[0], -1)

y_train = train.iloc[:, 0].values - 1 # subtract 1 to make the labels 0-based

: 

Test out different kinds of models and find the most effective architectures.|

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=10000)
model.fit(x_train, y_train)
y_pred = model.predict(x_train)

NameError: name 'x_train' is not defined

In [ ]:
# load a previous model
import joblib
model = joblib.load("./sklearn_models/logisticRegression.pkl")
y_pred = model.predict(x_train)

Perform hyperparameter tuning on best 3 models.

In [ ]:
from sklearn.model_selection import GridSearchCV

# or perform hyperparameter tuning
# Create the hyperparameters grid
param_grid = {

}

grid_search = GridSearchCV(())

# Train
grid_search.fit(x_train, y_train)

print(grid_search.best_params_)

# Use best model
model = grid_search.best_estimator_

In [ ]:
# get the training error
f1_score(y_train, y_pred, average='macro')
# print for the train set f1 score is
print("Train Set: " + str(f1_score(y_train, y_pred, average='macro')))

In [6]:
# get the prediction for the test set
test = pd.read_csv('./raw_data/balancedtest.csv')

# Preprocess the test set
preprocessed_test = [preprocess_text(text) for text in test.iloc[:, 1]]

X_test = np.array([vectorize_text(text) for text in preprocessed_test]) 
y_test = test.iloc[:, 0].values - 1 # subtract 1 to make the labels 0-based
y_pred = model.predict(X_test)

# get the f1 score
f1_score_test = f1_score(y_test, y_pred, average='macro')
print(f1_score_test)

NameError: name 'vectorize_text' is not defined

In [ ]:
# save the sklearn model
import joblib
joblib.dump(model, './sklearn_models/logisticRegression ' + str(f1_score_test) + 'tfidf.pkl')